# Text Klassifikation - Sentiment Analyse

In diesem Notebook möchten wir uns mit der Klassifikation von Texten beschäftigen. Vereinfacht gesagt beschäftigt sich Sentiment Analysis damit, natürlichsprachliche Aussagen dahingehend zu bewerten, ob die subjektive Aussage des Sprechers positiv oder negativ wertend gemeint ist.

Zu diesem Zweck haben wir den Datensatz von _Sentiment140_, einem Projekt der Stanford University, ausgewählt. Er beinhaltet 16 Millionen Tweets, die aufgrund der enthaltenen Emoticons automatisch in positiv und negativ eingeteilt wurden.

---

Dieses Notebook gliedert sich in die folgenden Teile:

1. Datenanalyse und Preprocessing
2. Tokenisierung & Vokabular
3. Klassifikation
4. Klassifikation mit huggingface transformers


In [1]:
import tensorflow as tf

tf.__version__


2022-11-15 10:19:07.091840: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


'2.10.0'

## 0. Vorbereitung


Als erstes muss der Datensatz von http://help.sentiment140.com/for-students in diese Colab Virtual Machine geladen werden.


In [144]:
DATA_DIR = "./download/sentiment140"

!mkdir -p {DATA_DIR}


In [145]:
from gdown import download
import os.path

if not os.path.isfile(
    "./download/sentiment140/testdata.manual.2009.06.14.csv"
) or not os.path.isfile(
    "./download/sentiment140/training.1600000.processed.noemoticon.csv"
):
    download(id="0B04GJPshIjmPRnZManQwWEdTZjg", output="./download/sentiment140.zip")


Downloading...
From: https://drive.google.com/uc?id=0B04GJPshIjmPRnZManQwWEdTZjg
To: /Users/sschwarzer/Desktop/repositories/ai-lab/nlp/download/sentiment140.zip
100%|██████████| 81.4M/81.4M [00:03<00:00, 26.8MB/s]


In [146]:
if not os.path.isfile(f"{DATA_DIR}/testdata.manual.2009.06.14.csv") or not os.path.isfile(f"{DATA_DIR}/training.1600000.processed.noemoticon.csv"): 
    !unzip -n {DATA_DIR}.zip -d {DATA_DIR}
    !rm {DATA_DIR}.zip
    !ls -la {DATA_DIR}

Archive:  ./download/sentiment140.zip
  inflating: ./download/sentiment140/testdata.manual.2009.06.14.csv  
  inflating: ./download/sentiment140/training.1600000.processed.noemoticon.csv  
total 497688
drwxr-xr-x  4 sschwarzer  staff        128 Nov 15 11:23 .
drwxr-xr-x  5 sschwarzer  staff        160 Nov 15 11:23 ..
-rw-r--r--  1 sschwarzer  staff      74326 Mar  4  2010 testdata.manual.2009.06.14.csv
-rw-r--r--  1 sschwarzer  staff  238803811 Mar  4  2010 training.1600000.processed.noemoticon.csv


## 1. Datenbeschaffung und -Analyse

Im ersten Schritt sollen folgende Schritte durchgeführt werden.


In [5]:
import pandas as pd
import os


1. Ladet den Datensatz in einen Pandas Dataframe. Welche Feature gibt es? Wie viele Samples gibt es?


> The data is a CSV with emoticons removed. Data file format has 6 fields:
>
> 1. the polarity of the tweet (0 = negative, 2 = neutral, 4 = positive)
> 2. the id of the tweet (2087)
> 3. the date of the tweet (Sat May 16 23:58:44 UTC 2009)
> 4. the query (lyx). If there is no query, then this value is NO_QUERY.
> 5. the user that tweeted (robotickilldozr)
> 6. the text of the tweet (Lyx is cool)


The dataset has 1.6 million samples.


2. Da wir uns nur für die Felder `polarity` und `text` interessieren, sollte die Liste mit den Daten folgendes Format haben : `id => (polarity, text)`.


In [6]:
# http://help.sentiment140.com/for-students
df = pd.read_csv(
    rf"{DATA_DIR}/training.1600000.processed.noemoticon.csv",
    encoding="latin-1",
    index_col=1,
    usecols=[0, 1, 5],
    names=["polarity", "id", "text"],
    converters={0: int, 1: int, 5: str},
)  # TODO


3. Wandelt diese Werte in 1 (positiv) und 0 (negativ).


In [7]:
df["polarity"].unique()


array([0, 4])

In [8]:
df["polarity"] = df["polarity"] / 4


In [9]:
df = df.astype({"text": str})


In [10]:
df.head()


,polarity,text
id,,
1467810369,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1467810672,0,is upset that he can't update his Facebook by ...
1467810917,0,@Kenichan I dived many times for the ball. Man...
1467811184,0,my whole body feels itchy and like its on fire
1467811193,0,"@nationwideclass no, it's not behaving at all...."


4. Fügt eine Spalte für die Anzahl an Wörtern im Text hinzu.


In [11]:
df["word_count"] = df["text"].str.strip().str.split().str.len()  # TODO
print(f'Mean tweet length: {df["word_count"].describe()["mean"]}')


Mean tweet length: 7.290991875


5. Analysiert den Datensatz mit den Pandas Boardmitteln.


In [12]:
# check whether all text was countable
df.isna().sum()


polarity      0
text          0
word_count    0
dtype: int64

In [13]:
df.describe()


,polarity,word_count
count,1600000.0,1.600000e+06
mean,0.5,7.290992e+00
std,0.5,8.037034e+00
min,0.0,1.000000e+00
25%,0.0,1.000000e+00
50%,0.5,1.000000e+00
75%,1.0,1.300000e+01
max,1.0,5.700000e+01


## 2. Tokenisierung

Um einen Einblick in die Daten zu bekommen und um später ein Modell zur Sentiment Analyse trainieren zu können, sollen die Daten nun aufbereitet werden. Da die sinntragenden Elemente in den Tweets die Wörter sind, sollten Sie die Tweets in Wörter aufteilen. Um genau zu sein, ist der Term 'Wörter' hier aus linguistischer Sicht etwas falsch, man spricht eigentlich von Tokens. Daher nennt man das Aufteilen von Text auch Tokenizing und die Funktion, die sowas kann, Tokenizer.

1. Der allereinfachste Tokenizer ist vermutlich die `split` Methode. Tokenisiert damit die eingelesen Tweets. Am Ende solltet ihr eine Liste `tokenized = [(polarity, [token_1,token_2, ...])]` erhalten.


In [14]:
df["tokenized"] = df["text"].str.split()  # TODO


Abgesehen von natürlichsprachlichen Wörtern sind in Tweets mindestens auch Hashtags, Mentions und Links enthalten. Überlegt euch, ob es Sinn ergibt, alle diese Bestandteile in den Daten in dieser From zu behalten. Begründet kurz Ihre Entscheidungen.
Falls ihr euch entschlossen habt, nicht alle diese Bestandteile zu behalten, filtert dementsprechend eure Daten. Die Struktur Ihrer Daten sollte am Ende gleich bleiben: `df['cleaned'] = [token_1,...]`


In [15]:
import re

expression = re.compile(
    r"[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)"
)
df["stripped"] = df["tokenized"].apply(
    lambda tokens: [
        "USER_TOKEN"
        if str.startswith(token, "@")
        else "HASHTAG_TOKEN"
        if str.startswith(token, "#")
        else "URL_TOKEN"
        if bool(expression.search(token))
        else token
        for token in tokens
    ]
)  # TODO


Zählt die Tokens in eurem Datensatz. Benutzt dafür ein Dictionary oder andere Python Build-Ins.


In [16]:
value_counts = pd.value_counts([token for tokens in df["stripped"] for token in tokens])


1. Wieviele unterschiedliche Wörter gibt es?


In [17]:
len(value_counts)


459777

2. Gebt die 100 häufigsten Wörter sortiert aus.


In [18]:
value_counts.index[:100]


Index(['1', 'USER_TOKEN', 'to', 'I', 'the', 'a', 'my', 'i', 'and', 'is', 'in',
       'URL_TOKEN', 'for', 'it', 'of', 'have', 'on', 'you', 'so', 'me', 'but',
       'that', 'not', 'at', 'be', 'I'm', 'was', 'just', 'with', 'get', 'this',
       'go', 'up', 'like', 'out', 'no', 'all', 'work', 'are', 'now', 'got',
       'do', 'going', 'don't', 'miss', 'want', 'from', 'day', 'really',
       'can't', 'back', '-', 'too', 'am', 'about', 'had', 'still', 'im',
       'it's', 'one', 'know', 'feel', '2', 'will', 'when', 'its', 'good',
       'think', 'need', 'last', 'your', 'My', 'today', 'has', 'time', 'see',
       'u', 'what', 'we', 'been', 'HASHTAG_TOKEN', 'sad', '&amp;', 'wish',
       'can', 'some', 'they', 'as', 'an', 'home', 'bad', 'more', 'much', 'i'm',
       'if', 'how', 'only', 'hate', 'her', 'off'],
      dtype='object')

Was zieht ihr aus den beiden Analysen? Was müsst ihr zusätzlich noch filtern?


> Stopwörter sollten auch noch gefiltert werden. Je nach dem kommt es außerdem auf den gewünschten Task am Ende an.


Eine Möglichkeit für komplexere Preprocessing-Methoden ist das Entfernen von Stoppwörtern. Hiefür nutzen wir NLTK.

Filtert eure gesäuberten Tokens auf Stopwörter.


In [19]:
import nltk


In [20]:
# df['cleaned'] = nltk.word_tokenize(df["text"]) # TODO
tweet_tokenizer = nltk.TweetTokenizer()
df["cleaned"] = df["text"].apply(tweet_tokenizer.tokenize)


In [21]:
df["cleaned"]


id
1467810369    [@switchfoot, http://twitpic.com/2y1zl, -, Aww...
1467810672    [is, upset, that, he, can't, update, his, Face...
1467810917    [@Kenichan, I, dived, many, times, for, the, b...
1467811184    [my, whole, body, feels, itchy, and, like, its...
1467811193    [@nationwideclass, no, ,, it's, not, behaving,...
                                    ...                        
2193601966                                                  [1]
2193601969                                                  [1]
2193601991                                                  [1]
2193602064                                                  [1]
2193602129                                                  [1]
Name: cleaned, Length: 1600000, dtype: object

## 3. Klassifikation

Wie Eingangs erwähnt, beschäftigt sich Sentiment Analysis damit, eine Äußerung automatisch dahingehend zu klassifizieren,
ob der Inhalt positiv oder negativ gemeint ist.
Es handelt sich also um eine binäre Klassifikation.

Für das Training des neuronalen Netzes orintiert sich der nachfolgende Teil an _Keras_ als Framework. Ihr könnt aber auch ein anderes Framework wie bspw. _Pytorch_ benutzen.

Aktuell liegen unsere Daten zwar in tokenisierter und gesäuberter Form vor, wir müssen unsere Daten aber noch in Vektoren transformieren.

Für die erste Klassifikation encoden wir die Eingabe als **Bag-of-Words**, sodass jedes potentielle Wort einem Eingabeneuron einspricht. _Ein Beispiel_: Zwei Tweets "lorem ipsum" und "foo foo bar", die Vektoren hätten die Länge 4 und für den ersten Tweet wäre der Vektor [1,1,0,0], für den zweiten [0,0,2,1].


### 3.1 Vokabular

Befüllt das dictionary `word2idx` so, dass jedes Wort auf einen Index abgebildet wird und die Indizes streng monoton aufsteigend sind. Für das Beispiel oben wäre `word2idx = {"lorem": 0, "ipsum": 1, "foo": 3, "bar": 4}`


In [22]:
df


,polarity,text,word_count,tokenized,stripped,cleaned
id,,,,,,
1467810369,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",19,"[@switchfoot, http://twitpic.com/2y1zl, -, Aww...","[USER_TOKEN, URL_TOKEN, -, Awww,, that's, a, b...","[@switchfoot, http://twitpic.com/2y1zl, -, Aww..."
1467810672,0,is upset that he can't update his Facebook by ...,21,"[is, upset, that, he, can't, update, his, Face...","[is, upset, that, he, can't, update, his, Face...","[is, upset, that, he, can't, update, his, Face..."
1467810917,0,@Kenichan I dived many times for the ball. Man...,18,"[@Kenichan, I, dived, many, times, for, the, b...","[USER_TOKEN, I, dived, many, times, for, the, ...","[@Kenichan, I, dived, many, times, for, the, b..."
1467811184,0,my whole body feels itchy and like its on fire,10,"[my, whole, body, feels, itchy, and, like, its...","[my, whole, body, feels, itchy, and, like, its...","[my, whole, body, feels, itchy, and, like, its..."
1467811193,0,"@nationwideclass no, it's not behaving at all....",21,"[@nationwideclass, no,, it's, not, behaving, a...","[USER_TOKEN, no,, it's, not, behaving, at, all...","[@nationwideclass, no, ,, it's, not, behaving,..."
...,...,...,...,...,...,...
2193601966,1,1,1,[1],[1],[1]
2193601969,1,1,1,[1],[1],[1]
2193601991,1,1,1,[1],[1],[1]


In [23]:
word2idx = {
    word: index
    for index, word in enumerate(
        pd.value_counts([token for tokens in df["cleaned"] for token in tokens]).index
    )
}  # TODO
word2idx


{'1': 0,
 '.': 1,
 '!': 2,
 'to': 3,
 'I': 4,
 'the': 5,
 ',': 6,
 'a': 7,
 'my': 8,
 'i': 9,
 '...': 10,
 'and': 11,
 'is': 12,
 '?': 13,
 'in': 14,
 'it': 15,
 'for': 16,
 'of': 17,
 'me': 18,
 'you': 19,
 'on': 20,
 'have': 21,
 'so': 22,
 'but': 23,
 'that': 24,
 'not': 25,
 'at': 26,
 'be': 27,
 "I'm": 28,
 'was': 29,
 '..': 30,
 'just': 31,
 'with': 32,
 'up': 33,
 'this': 34,
 'now': 35,
 'get': 36,
 'go': 37,
 'work': 38,
 'out': 39,
 'no': 40,
 'like': 41,
 'all': 42,
 'day': 43,
 '-': 44,
 'today': 45,
 'too': 46,
 'do': 47,
 'are': 48,
 'going': 49,
 'got': 50,
 'back': 51,
 "don't": 52,
 'miss': 53,
 'want': 54,
 'really': 55,
 'from': 56,
 "can't": 57,
 'am': 58,
 '"': 59,
 'know': 60,
 'time': 61,
 'about': 62,
 'good': 63,
 'sad': 64,
 'had': 65,
 'one': 66,
 'still': 67,
 'im': 68,
 "it's": 69,
 'home': 70,
 '2': 71,
 '&': 72,
 'u': 73,
 'its': 74,
 'think': 75,
 'will': 76,
 'feel': 77,
 'when': 78,
 'what': 79,
 'need': 80,
 'bad': 81,
 'last': 82,
 'My': 83,
 'see': 

Welche Länge werden die Vektoren haben?


In [24]:
VECTOR_LEN = len(word2idx)  # TODO
print(VECTOR_LEN)


466230


Wir könnten mit `numpy` ein Array befüllen, das für jeden der 16 Millionen Tweets einen Vektor wie oben beschrieben enthält.
Bevor ihr damit beginnen, überschlagt, wieviel Speicherplatz (im Hauptspeicher) ein solches Array belegen würde, wenn jeder Eintrag 32 bit hat. Reicht euer Hauptspeicher dafür aus?


In [25]:
MEMORY = VECTOR_LEN * 16_000_000 * 32  # TODO
print(f"Das numpy Array würde {(MEMORY / 8 / (1024 **4)):.2f} TB benötigen")


Das numpy Array würde 27.14 TB benötigen


### 3.2 Data Generator

Um das Problem mit dem zu kleinen Hauptspeicher zu umgehen, bietet Keras die Möglichkeit, anstatt auf einem kompletten Datensatz zu operieren, immer nur kleinere Häppchen abzuarbeiten. Dazu wird ein Python-Generator eingesetzt.
Vervollständigt die Funktion unten, so dass ein Generator entsteht. Die Parameter der Funktion sind:

- x, y: tokenisierte und gesäuberte Tweets und Labels
- w2i: das word2index dictionary
- batch_size: Anzahl der vektorisierten Tweets, die pro Aufruf zurückgegeben werden sollen.

Die benutzen Tweets nacheinander aus `x, y` gewählt werden und kein Tweet mehrfach zurückgegeben werden.


In [42]:
import random
import numpy as np


def data_generator(x, y, w2i, batch_size):
    num_batches = int(np.ceil(len(x) / batch_size))  # TODO
    current_batch = 0

    while True:
        # Initialize with zeros
        batch_x = np.zeros((batch_size, len(w2i.keys())), dtype=np.int64)
        batch_y = np.zeros((batch_size, 1))

        # TODO
        # Iterate batches
        start_index = current_batch * batch_size
        for i in range(batch_size):
            if i + start_index >= len(x):
                break
            # Add to _batch_x_ at index of _np.vectorize_ value _1_ for each found index
            np.add.at(
                batch_x[i],
                np.vectorize(
                    lambda word: w2i.get(word)
                    if w2i.get(word) != None
                    else -1  # return -1 if word is not in vocab
                )(x[i + start_index]),
                1,
            )
            batch_y[i] = y[i]

        current_batch += 1
        yield batch_x, batch_y

        # Reset counter
        if current_batch >= num_batches:
            current_batch = 0


In [27]:
BATCH_SIZE = 32

gen = data_generator(
    df["cleaned"].to_numpy(), df["polarity"].to_numpy(), word2idx, BATCH_SIZE
)  # TODO


Ihr könnt euren Generator wie folgt ausprobieren:


In [28]:
next(gen)


(array([[0, 2, 0, ..., 0, 0, 0],
        [0, 1, 1, ..., 0, 0, 0],
        [0, 1, 0, ..., 0, 0, 0],
        ...,
        [0, 1, 0, ..., 0, 0, 0],
        [1, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]),
 array([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]]))

### 3.3 Neuronales Netz

Wir sind nun endlich soweit, unser neuronales Netz aufzubauen. Da unser Netz genau ein hidden Layer hat und auch sonst nicht sonderlich komplex ist, benutzen wir die _Sequential_-API von Keras.


In [29]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

m = Sequential()


2022-11-15 10:20:55.719714: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Fügt einen _Dense_-Layer dem Netz hinzu, als _hidden units_ könnt ihr 16 nehmen. Da dies auch der Eingabe-Layer ist, müsst ihr den Parameter `input_shape` definieren. (Siehe auch: https://keras.io/layers/core/)


In [30]:
m.add(Dense(16, input_shape=(VECTOR_LEN), activation="relu"))  # TODO


Als letzten Layer in unserem neuronalen Netz, fügt einen weiteren _Dense_-Layer hinzu. Dieser Layer dient auch als "Ausgabelayer" Überlegt euch die Anzahl der _hidden units_ (Hinweis: Wie lässt sich unser Machine-Learning-Problem kategorisieren?) Welche _Activation_-Funktion wählt ihr?


In [31]:
m.add(Dense(1, activation="sigmoid"))  # TODO


7. Kompiliert das neuronale Netz. Als `optimizer` könnt ihr 'adam' benutzen. Wählt eine passende `loss`-Funktion aus. Begründet eure Entscheidung. (https://keras.io/models/model/#compile)


In [32]:
# TODO
m.compile(optimizer="adam", loss="binary_crossentropy")


### 3.3. Training

Bevor ihr nun das neuronale Netz trainiert, teilt noch euren Datensatz in zwei Teile auf. Einen Teil zum Trainieren und einen zum Evaluieren. Das Verhältnis der beiden Datensätze sollte 70%:30% sein. Bevor ihr die Daten aufteilen, durchmischt sie mit der `shuffle`-Methode aus dem `random`-Modul. Außerdem solltet ihr die Datenmenge zunächst auf ca. 100000 begrenzen, damit das Training des neuronalen Netzes nicht ewig dauert.


In [33]:
# TODO
from sklearn.model_selection import train_test_split

SAMPLE_SIZE = 100000
sample_df = df.sample(SAMPLE_SIZE)

X_train_sample, X_val_sample, y_train_sample, y_val_sample = train_test_split(
    sample_df["cleaned"].to_numpy(), sample_df["polarity"].to_numpy(), test_size=0.3
)


Wir sind nun soweit das neuronale Netz zu trainieren. Da wir den oben entwickelten Generator einsetzen wollen, verwenden wir dazu die `fit_generator`-Methode. Als `batch_size` könnt ihr 100 nehmen, für den `epochs`-Parameter 10. Was wählt ihr als `steps_per_epoch`-Parameter? (https://keras.io/models/model/#fit_generator)


In [34]:
BATCH_SIZE = 100
PATIENCE = 3

train_gen_sample = data_generator(X_train_sample, y_train_sample, word2idx, BATCH_SIZE)
val_gen_sample = data_generator(X_val_sample, y_val_sample, word2idx, BATCH_SIZE)

train_steps_sample = int(np.ceil(len(X_train_sample) / BATCH_SIZE))
val_steps_sample = int(np.ceil(len(X_val_sample) / BATCH_SIZE))

model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    "./download/sentiment140/model_checkpoint_sample.hd5", save_best_only=True
)
early_stopping = tf.keras.callbacks.EarlyStopping(
    verbose=1, patience=PATIENCE, restore_best_weights=True
)

m.fit(
    x=train_gen_sample,
    validation_data=val_gen_sample,
    steps_per_epoch=train_steps_sample,
    validation_steps=val_steps_sample,
    epochs=10,
    callbacks=[model_checkpoint, early_stopping],
)  # TODO
print(m.summary())


Epoch 1/10
700/700 [==============================] - 187s 266ms/step - loss: 0.6932 - val_loss: 0.6999
Epoch 2/10
700/700 [==============================] - 191s 273ms/step - loss: 0.6545 - val_loss: 0.7328
Epoch 3/10
700/700 [==============================] - 193s 275ms/step - loss: 0.5772 - val_loss: 0.8046
Epoch 4/10
700/700 [==============================] - 195s 279ms/step - loss: 0.5139 - val_loss: 0.8812
Epoch 4: early stopping
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 16)                7459696   
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 7,459,713
Trainable params: 7,459,713
Non-trainable params: 0
_________________________________________________________________
No

Während das Netz trainiert wird, könnt ihr euch Gedanken zur Evaluierung machen:


- Definiert die üblichen Fehlerklassen (wahr positiv, falsch positiv, wahr negativ, falsch negativ)


- Eine häufig benutzte Evaluationsmetrik ist die _Accuracy_. Beschreibt diese Metrik und schreibt die Formel zur Berechnung auf.


> Accuracy gibt die Treffergenauigkeit aller Vorhersagen zu ihrem wahren Wert an.
> $$\text{Accuracy} = \frac{TP + TN}{TP + TN + FP + FN} \Leftrightarrow \frac{TP + TN}{\text{All Predictions}}$$


- Warum könnte die _Accuracy_ eine schlechte Metrik sein?


> Weil sie relativ anfällig für unbalancierte Datasets ist.


- Zur Evaluation von binären Klassifikationsproblemen wird in der Literatur gerne _Precision_ und _Recall_ verwendet. Wie sind die beiden Evaluationsmaße definiert? Beschreibt diese Metriken mit eigenen Worten. Schreibt auch die Formeln zur Berechnung auf.


> Precision und Recall zeichnen sich durch die unten beschriebenen Formeln aus.
> Die Precision gibt dabei den Anteil der korrekt vorhergesagten aus allen als Positiv vorhergesagten Daten wieder, wohingegen der Recall wie Wahrscheinlichkeit an mit der ein als positiv vorhergesagtes Datum wirklich positiv ist.  
> $$\text{Precision} = \frac{TP}{TP + FP}$$
> $$\text{Recall} = \frac{TP}{TP + FN}$$


- Warum könnten _Precision_ und _Recall_ bessere Metriken sein als _Accuracy_?


> Es wird der Fehler 1. Art und 2. Art separat betrachtet.


11. Inzwischen sollte das Netz fertig trainiert sein. Speichert es ab!


In [35]:
m.save("my_net_sample.h5")


### 3.4. Evaluation

Evaluiert euer Netz mit dem Datensatz, den ihr oben beseite gelegt haben. Benutzt dafür die `predict`-Methode des Models. Berechnet dafür _Precision_, _Recall_, _Accuracy_ und _Confusion Matrix_.

Interpretiert kurz eure Ergebnisse.


In [104]:
# TODO
df_test = pd.read_csv(
    rf"{DATA_DIR}/testdata.manual.2009.06.14.csv",
    encoding="latin-1",
    index_col=1,
    usecols=[0, 1, 5],
    names=["polarity", "id", "text"],
    converters={0: int, 1: int, 5: str},
)

df_test = df_test.loc[
    (df_test["polarity"] == 1) | (df_test["polarity"] == 0)
]  # Remove polarity 2 from the dataset

df_test["polarity"] = df_test["polarity"] / 4
df_test = df_test.astype({"text": str})

df_test["cleaned"] = df_test["text"].apply(tweet_tokenizer.tokenize)

test_gen = data_generator(
    df_test["cleaned"].to_numpy(), df_test["polarity"].to_numpy(), word2idx, BATCH_SIZE
)

STEPS = 1
MAX_STEPS = min(STEPS, int(np.ceil(len(df_test) / BATCH_SIZE)))

predictions = m.predict(test_gen, batch_size=BATCH_SIZE, steps=MAX_STEPS)

binary_predictions = np.round(predictions)


1/1 [==============================] - 0s 349ms/step


In [141]:
def get_category(truth, prediction):
    if truth == prediction == 1:
        return "TP"
    elif truth == prediction == 0:
        return "TN"
    elif truth != prediction == 1:
        return "FP"
    elif truth != prediction == 0:
        return "FN"


evaluation = pd.DataFrame()
evaluation["truth"] = df_test["polarity"][: (MAX_STEPS * BATCH_SIZE)].to_numpy()
evaluation["prediction"] = binary_predictions
evaluation["category"] = evaluation.apply(
    lambda row: get_category(row["truth"], row["prediction"]), axis=1
)


In [157]:
cfm = evaluation["category"].value_counts(normalize=True)


In [163]:
precision = cfm.get("TP", 0.) / (cfm.get("TP", 0.) + cfm.get("FP", 0.))
recall = cfm.get("TP", 0.) / (cfm.get("TP", 0.000000001) + cfm.get("FN", 0.))
print(f"precision: {precision}\nrecall: {recall}\n🤡")

precision: 0.0
recall: 0.0
🤡


## 3b. (optional) Erweiterte Klassifikation

Im nächsten Schritt wollen wir das Netz auf dem vollen Datensatz trainieren und die Komplexität etwas erhöhen.

1. Trainiert euer Netz auf dem großen Datensatz.


In [164]:
# TODO
X_train, X_val, y_train, y_val = train_test_split(
    df["cleaned"].to_numpy(), df["polarity"].to_numpy(), test_size=0.3
)

train_gen = data_generator(X_train, y_train, word2idx, BATCH_SIZE)
val_gen = data_generator(X_val, y_val, word2idx, BATCH_SIZE)

train_steps = int(np.ceil(len(X_train) / BATCH_SIZE))
val_steps = int(np.ceil(len(X_val) / BATCH_SIZE))

model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    "./download/sentiment140/model_checkpoint_full.hd5", save_best_only=True
)
early_stopping = tf.keras.callbacks.EarlyStopping(
    verbose=1, patience=PATIENCE, restore_best_weights=True
)

m.fit(
    x=train_gen,
    validation_data=val_gen,
    steps_per_epoch=train_steps,
    validation_steps=val_steps,
    epochs=10,
    callbacks=[model_checkpoint, early_stopping],
)
print(m.summary())


Epoch 1/10
   37/11200 [..............................] - ETA: 39:43 - loss: 0.6956

KeyboardInterrupt: 

2. Verändert die Parameter Ihres Netzes (z.B Anzahl _hidden units_, Anzahl _hidden layers_) und trainiert das Netz erneut (auf dem kleinen Datensatz). Was stellt ihr fest?


In [165]:
train_gen_sample = data_generator(X_train_sample, y_train_sample, word2idx, BATCH_SIZE)
val_gen_sample = data_generator(X_val_sample, y_val_sample, word2idx, BATCH_SIZE)

m_wide = Sequential()
m_wide.add(Dense(64, input_shape=(VECTOR_LEN,)))
m_wide.add(Dense(1, activation="sigmoid"))
m_wide.compile(optimizer="adam", loss="binary_crossentropy")

m_wide.fit(
    x=train_gen_sample,
    validation_data=val_gen_sample,
    steps_per_epoch=train_steps_sample,
    validation_steps=val_steps_sample,
    epochs=10,
    callbacks=[model_checkpoint, early_stopping],
)  # TODO
print(m_wide.summary())


NameError: name 'X_train_sample' is not defined

In [ ]:
train_gen_sample = data_generator(X_train_sample, y_train_sample, word2idx, BATCH_SIZE)
val_gen_sample = data_generator(X_val_sample, y_val_sample, word2idx, BATCH_SIZE)

m_deep = Sequential()
m_deep.add(Dense(16, input_shape=(VECTOR_LEN,)))
m_deep.add(Dense(16))
m_deep.add(Dense(16))
m_deep.add(Dense(16))
m_deep.add(Dense(1, activation="sigmoid"))
m_deep.compile(optimizer="adam", loss="binary_crossentropy")

m_deep.fit(
    x=train_gen_sample,
    validation_data=val_gen_sample,
    steps_per_epoch=train_steps_sample,
    validation_steps=val_steps_sample,
    epochs=10,
    callbacks=[model_checkpoint, early_stopping],
)  # TODO
print(m_deep.summary())


## 4. Sentiment Analyse mit Huggingface Transformers

In diesem Abschnitt werden wir nun Huggingface (https://huggingface.co/) zu Sentence Classification nutzen.

Hierfür benötigen wir die Libraries `datasets` (für preprocessing) und `transformers` (für die Transformer Modelle). Ebenfalls werden wir das Tokenisierung nicht mehr selbst durchführen, sondern viel mehr die `tokenizers` von huggingface hierfür benutzen.

Für eine gute Einführung in Klassifikation mit Transformers siehe: https://huggingface.co/docs/transformers/tasks/sequence_classification


In [ ]:
!pip install datasets transformers

### 4.1 Data Preparation

Bevor wir ein huggingface transformer Modell nutzen können, müssen wir:

- den Datensatz (`df`) als `Dataset` laden
- ladet zunächst nur einen Teil des Datensatzes und wenn das Training funktioniert, vergrößert den Datensatz.
- die Spalten umbennenen
- Train / Test Split (90 / 10)


1. Lade den Datensatz


In [ ]:
dataset = # TODO

2. Entferne alle unnötigen Spalten und benennen `polarity` in `labels` um. Achte darauf, dass die Spalte `labels` vom Feature-Typ `ClassLabel` ist.


In [ ]:
dataset = # TODO

In [ ]:
features = # TODO
dataset = dataset.cast(features)

3. Teilt den Datensatz in train und test split auf.


In [ ]:
dataset = # TODO

Schaut euch die Klassenverteilung im Datensatz an? Habt ihr noch eine 50/50 Verteilung zwischen positiv und negativ?


### 4.1. Training

Nun haben wir den Datensatz geladen und vorbereitet.

1. Ladet ein `AutoModelForSequenceClassification` eurer Wahl (https://huggingface.co/models) und den entsprechenenden `AutoTokenizer`.

- Welches Modell nutzt ihr? Auf welchen Daten wurde das Netz vortrainiert?
- Auf welchem Framework basiert das Modell?


In [ ]:
tokenizer = # TODO

In [ ]:
model = # TODO

2. Bevor das eigentliche Training beginnt, muss der Datensatz noch tokenized werden. Nutzt hierfür den oben instantiierten `tokenizer` und beantwortet folgende Fragen:

- Wie groß ist das des Tokenizers?
- In welcher Granularität werden Texte aufgesplittet?


In [ ]:
tokenized_dataset = # TODO

3. Baut euch eine Methode `compute_metrics`, die die notwendigen Metriken während des Trainings sammelt. Nutzt hierfür wieder Precision, Recall und F1 score.


In [ ]:
def compute_metrics(eval_pred):
    """Computes the metrics given a tuple of (logits, labels)"""

    # TODO
    return {
        "accuracy": accuracy,
        "f1": f1,
        "precision": precision,
        "recall": recall,
    }


4. Ladet einen `Trainer` und spezifiziert die `TrainingArguments`. Es bietet sich an, nach jeder Epoche zu evaluieren und das beste Modell am Ende zu laden.


In [ ]:
finetuning_args = TrainingArgument()  # TODO
trainer = Trainer()  # TODO


5. Trainiert das Modell und behaltet die Metriken im Auge.


In [ ]:
trainer.train()


### 4.3. (Optional) Training mit weiterem Modell

Falls ihr möchtet, könnt ihr optional das geladene vortrainierte Modell austauschen und ein anderes (größeres, auf anderen Daten trainiert, ...) Modell nutzen.


### 4.4. Prediction

Nun könnt ihr das Modell nutzen, um ein paar Beispiel aus dem Test-Datensatz zu predicten.


In [ ]:
id2label = {0: "neg", 1: "pos"}
prediction_ds = tokenized_dataset["test"].select(range(50))


Gebt die falschen Predictionn aus. Was fällt euch auf?


In [ ]:
raw_pred, _, _ = trainer.predict(prediction_ds)
print(f"Total predictions: {len(raw_pred)}")

for text, prediction in zip(prediction_ds, np.argmax(raw_pred, axis=1)):
    expected = text["labels"]
    if expected != prediction:
        print(
            f"Text: {text['text']} --- Expected: {id2label[expected]} --- Predicted: {id2label[prediction]}"
        )
